## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [27]:
def ct(file:str,chap_num:int,num:int,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    with open(f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}","r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [28]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        result=query.fetchall()
    return result

## List

In [29]:
ct("List",1,1)
exe("select 出金額 from 家計簿")

[(380,), (0,), (2800,), (5000,), (7560,)]